# Implied Volatility


## Import Required Libraries

In [1]:
# Data Manipulation
import numpy as np
from datetime import datetime

# Import blackscholes object
from src.optionpricer2 import BS
from tabulate import tabulate

# Implied Volatility Formulation

## Newton Method

In [2]:
def newton_iv(className, spot, strike, rate, dte, callprice=None, putprice=None):
    x0 = 1  # initial guess
    h = 0.001  # step size
    tolerance = 1e-7  # 7-digit accuracy is desired
    epsilon = 1e-14  # do not divide by a number smaller than this, some kind of error / floor
    maxiter = 200  # maximum number of iterations to execute
    # function whose root we are trying to find
    # f(x) = Black Scholes Call price - Market Price - defining the f(x) here
    if callprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).callPrice - callprice
    if putprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).putPrice - putprice
    for i in range(maxiter):
        y = f(x0)  # starting with initial guess
        yprime = (f(x0 + h) - f(x0 - h)) / (
            2 * h
        )  # central difference, the derivative of the function

        if abs(yprime) < epsilon:  # stop if the denominator is too small
            break
        x1 = x0 - y / yprime  # perform Newton's computation

        if abs(x1 - x0) <= tolerance * abs(
            x1
        ):  # stop when the result is within the desired tolerance
            break

        x0 = x1  # update x0 to start the process again

    return x1  # x1 is a solution within tolerance and maximum number of iterations

In [3]:
# newton iv
args = (100, 100, 0.02, 1)
vol = newton_iv("BS", *args, callprice=8)
vol

0.17657213831399154

In [4]:
# validate that it works
BS(100, 100, 0.02, 1, vol).callPrice

7.999999999999986

## Bisection Method

In [5]:
# Bisection Method
def bisection_iv(className, spot, strike, rate, dte, callprice=None, putprice=None, high=500.0, low=0.0):
    # this is market price
    if callprice:
        price = callprice
    
    if putprice and not callprice:
        price = putprice
    
    tolerance = 1e-7
    
    for i in range(1000):
        mid = (high + low) / 2 # c= (a+b)/2
        if mid < tolerance:
            mid = tolerance

        if callprice:
            estimate = eval(className)(spot, strike, rate, dte, mid).callPrice # Blackscholes price

        if putprice:
            estimate = eval(className)(spot, strike, rate, dte, mid).putPrice

        if round(estimate,6) == price:
            break
        elif estimate > price:
            high = mid # replace c with b | b = c
        elif estimate < price:
            low = mid # replace c with a | a = c
    return mid

In [6]:
# bisection iv
vol = bisection_iv('BS', *args, callprice=8)
vol

0.17657213902566582

In [7]:
# validate that it works
BS(100, 100, 0.02, 1, vol).callPrice

8.000000027820803

# BS Implied Volatility

In [8]:
# Initialize option
call_by_price = BS(100,100,0.05,1, callprice=10)
call_by_vol = BS(100,100,0.05,1, volatility=.2)

header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', 'IV']
table = [
    [option.callPrice, option.callDelta, option.gamma, option.callTheta, option.vega, option.callRho, option.impvol]
    for option in [call_by_price, call_by_vol]
]

print(tabulate(table,header))

  Option Price     Delta     Gamma       Theta      Vega       Rho        IV
--------------  --------  --------  ----------  --------  --------  --------
       10       0.64057   0.019892  -0.0170332  0.373913  0.54057   0.187972
       10.4506  0.636831  0.018762  -0.0175727  0.37524   0.532325  0.2


# Reference

https://micropython-ulab.readthedocs.io/en/latest/scipy-optimize.html